Для начала, нам нужно написать функцию, которая "вытаскивает" данные о последовательности из файла расширеня .fasta. 

Для этого откроем файл и прочитаем всё, кроме первой (в терминах программистов - нулевой) строчки (которая содержит описание последовательности после знака >). Затем мы проверяем, что то, что мы читаем - биоинформатическая последовательность (только заглавные английские буквы), без цифр, астерисков и других символов. Данная функция представлена ниже.  

In [1]:
#Чтение fasta файлов
def fasta_to_seq(link):
    with open(link) as file:
        lines = file.readlines() #прочитали строчки и запомнили
    seq = ''
    lines = lines[1:]
    for i in range (len(lines)):
        if lines[i].isalpha() and lines[i].isupper(): #проверяем, что то, что мы добавляем - английская заглавная буква
            seq = seq + lines[i]
    return seq

Зададим все необходимые переменные:

1. Последовательности seq1 и seq2, которые мы "достали" из FASTA-файла;
2. Очки, которые мы назначаем совпадению (match), несовпадению (mismatch), и пропуску (gap). По условию задания штраф за прпуск в два раза меньше штрафа за несовпадение;
3. Пустые строки alignment1 и alignment2, которые в дальнейшем будут выводом итогового выравнивания

Также создадим нулевую матрицу (n+1) * (m+1), выделяя строку и столбец под сравнение с пропуском, где n = len(seq1), m = len(seq2).
Для проверки также сделаем output наших последовательностей.

In [2]:
seq1 = fasta_to_seq('1.fasta') 
seq2 = fasta_to_seq('2.fasta') 

match = 2 
mismatch = -2 
gap = -1 

alignment1 = '' 
alignment2 = ''

M = [[0 for _ in range(len(seq1) + 1)] for _ in range(len(seq2) + 1)] #задали матрицу со строчкой для пропусков

print('Последовательность 1:', seq1)
print('Последовательность 2:', seq2)

Последовательность 1: KNWEDFEIAAENMYMANPQNCRYTMKYVHSKGHILLKMSDNVKCVQYRAENMPDLKK
Последовательность 2: EEFSRAVEKLYLTDPMKVRVVLKYRHCDGNLCIKVTDNSVVSYEMRLFGVQKDNFALEHSLL


Дальше приступаем к заполнению матрицы схожести. Первым делом заполним нулевую строку и нулевой столбец, учитывая, что мы сравниваем предыдущий вертикальный/горизонтальный элемент с пропуском (назначая пенальти за gap). 

In [3]:
for i in range (1, len(seq2)+1):
    M[i][0] = M[(i-1)][0] + gap
for j in range (1, len(seq1)+1):
    M[0][j] = M[0][(j-1)] + gap

Продолжим заполнение матрицы схожести. 

Напомним, что ячейке присваивается то значение, которое является максимальным из трёх возможных m_ver ("приходим" в ячейку сверху, сравниваем с пропуском, суммируем значение предыдущей ячейки и gap), m_hor ("приходим" в ячейку слева, сравниваем опять же с пропуском, суммируем значение предыдущей ячейки и gap) и m_dia ("приходим" в ячейку слева сверху, сравниваем буквы в столбце и строке между собой, назначаем штраф за мисмэтч или мэтч, суммируем значение предыдущей ячейки и mismatch/match).

In [4]:
#заполним табличку
for i in range (1, len(seq2)+1):
    for j in range (1, len(seq1)+1):
        m_ver = M[i-1][j] + gap #приходим сверху - значение ячейки откуда пришли + гэп
        m_hor = M[i][j-1] + gap #приходим слева - значение ячейки откуда пришли + гэп      
        if seq2[i-1] == seq1[j-1]:
            m_dia = M[i-1][j-1] + match #диагональ если значения элементов равны
        else:
            m_dia = M[i-1][j-1] + mismatch #диагональ если значения элементов равны
            
        M[i][j] = max(m_ver, m_hor, m_dia) #заполним ячейку максимальным из трёх значений

Теперь нам нужно построить непосредственно выравнивание по матрице схожести. Для этого мы постепенно прослеживаем путь, с помощью которого мы пришли в самую правую нижнюю ячейку матрицы. 

Заведём счётчики равные длине строк, будем вычитать из них единицу каждый раз, когда делаем "шаг". Шаги будем продолжать до тех пор, пока счётчики не обнулятся, то есть мы не придём в самую первую ячейку.

Отслеживать путь мы будем сравнивая значение ij-той ячейки с соседом + назначенное пенальти (если пришли слева или сверху, то это gap, если по диагонали - то это мэтч или мисмэтч). Если сосед + пенальти в сумме даёт нашу ячейку, то значит, что мы пришли со стороны именно этого соседа.

Отдельного внимания заслуживает то, как мы поступаем с "диагональными" ячейками. Помимо сравнения ячейки, в которой мы находимся, с соседом + пенальти, мы также проверяем, равны ли буквы, чтобы если существует несколько путей, выбрать наиболее верный.

In [5]:
n = len(seq1) #счётчик шагов от длины строки до нуля
m = len(seq2)

while m != 0 or n != 0: 
    score = M[m][n] 
    score_ver = M[m-1][n]
    score_hor = M[m][n-1]
    score_dia = M[m-1][n-1]
    if score == score_dia + match and seq1[n - 1] == seq2[m - 1]:  
        alignment1 = alignment1 + seq1[n-1]                                
        alignment2 = alignment2 + seq2[m-1]
        m = m - 1
        n = n - 1
    elif score == score_ver + gap:
        alignment1 = alignment1 + '_'
        alignment2 = alignment2 + seq2[m-1] 
        m = m-1
    elif score == score_hor + gap: 
        alignment1 = alignment1 + seq1[n-1]
        alignment2 = alignment2 + '_'
        n = n-1
    elif score == score_dia + mismatch and seq1[n - 1] != seq2[m - 1]:
        alignment1 = alignment1 + seq1[n - 1]
        alignment2 = alignment2 + seq2[m - 1]
        m = m - 1
        n = n - 1
        
print('Итоговое выравнивание:')
print(alignment1[::-1]) #пишем в обратную сторону так как мы добавляли каждый новый символ в конец
print(alignment2[::-1])

Итоговое выравнивание:
KNWEDFEIA___A_ENMYMANPQNCR__Y_T__MKYVHS____KG__HIL____L__KMS__DN_VKCVQ_YRAENMPD_LK____K__________
___E__E__FSRAVE___________KLYLTDPMK_V__RVVLK_YRH__CDGNLCIK__VTDNSV__V_SY__E_M__RL_FGVQKDNFALEHSLL


Вот такая вот реализация алгоритма Нидлмана-Вунша выравнивания двух последовательностей!